In [2]:
import pyodbc
# Define the connection string
server = '172.10.2.5'
database = 'Car Store DB'
username = 'Carstore'
password = 'nss@12345'
connection_string = f'DRIVER={{SQL Server Native Client 11.0}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
# Function to calculate the EMI
def calculate_emi(principal, annual_interest_rate, years):
    interest_rate = annual_interest_rate / 12 / 100  # Monthly interest rate
    total_months = years * 12
    emi = principal * interest_rate * (1 + interest_rate) ** total_months / ((1 + interest_rate) ** total_months - 1)
    return emi
# Function to insert the EMI, tenure, and carid into the "Emi" table
def insert_emi(principal, annual_interest_rate, years, emi, carid, u_id,payment_id):
    try:
        connection = pyodbc.connect(connection_string)
        cursor = connection.cursor()
        # Insert calculated EMI, tenure, and carid into the "Emi" table
        cursor.execute(
            "INSERT INTO Emi (emi_amount_per_month, emi_tenure, carid, u_id,payment_id) VALUES (?, ?, ?, ?,?);",
            (emi, years, carid, u_id,payment_id)
        )
        connection.commit()
        connection.close()
    except Exception as e:
        print(f'Failed to insert data into Emi table: {str(e)}')
def main():
    try:
        connection = pyodbc.connect(connection_string)
        cursor = connection.cursor()
        # Fetch the car_price, bank_interest, and carid from the latest user's purchase timestamp
        cursor.execute("SELECT TOP 1 payment_id, car_price, bank_interest, carid, u_id FROM car_selection ORDER BY purchase_timestamp DESC")
        row = cursor.fetchone()
        if row:
            principal = row.car_price
            annual_interest_rate = row.bank_interest
            carid = row.carid
            u_id = row.u_id
            payment_id = row.payment_id
        else:
            print("No purchase history found.")
            return  # Exit the function if no payment history is found
    except Exception as e:
        print(f'Failed to fetch data: {str(e)}')
        return  # Exit the function if an exception occurs
    available_years = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Year options
    while True:
        try:
            print("Choose the number of years for EMI from the following options:")
            for i, years in enumerate(available_years, 1):
                print(f"{years} years")
            choice = int(input("Enter the number corresponding to your choice: "))
            if choice in range(1, len(available_years) + 1):
                years = available_years[choice - 1]
                break  # Exit the loop if a valid choice is made
            else:
                print("Invalid choice. Please select a valid option.")
        except ValueError:
            print("Invalid input. Please enter a valid choice.")
    emi = calculate_emi(principal, annual_interest_rate, years)
    if emi > 0:
        print(f"You have to pay an installment of ${emi:.2f} every month for {years} year(s).")
        # Call the insert_emi function to insert data into the Emi table
        insert_emi(principal, annual_interest_rate, years, emi, carid, u_id, payment_id)
    else:
        print("Invalid input. Please enter valid values for years.")
    try:
        connection = pyodbc.connect(connection_string)
        cursor = connection.cursor()
        # Fetch the latest car selection data and the corresponding Emi details
        cursor.execute("""
            SELECT TOP 1 cs.car_name, cs.car_price, cs.bank_name, cs.bank_interest, cs.payment_id, cs.u_id
            FROM car_selection cs
            WHERE cs.payment_method = 'EMI'
            ORDER BY cs.purchase_timestamp DESC
        """)
        row = cursor.fetchone()
        if row:
            car_name, car_price, bank_name, bank_interest, payment_id, u_id = row
            # Retrieve Emi details for the given payment_id
            cursor.execute("""
                SELECT emi_amount_per_month, emi_tenure
                FROM Emi
                WHERE payment_id = ? AND u_id = ?
            """, (payment_id, u_id))
            emi_row = cursor.fetchone()
            if emi_row:
                emi_amount_per_month, emi_tenure = emi_row
                # Create a DataFrame with the retrieved information
                data = {
                    "Car Name": [car_name],
                    "Car Price": [car_price],
                    "Bank Name": [bank_name],
                    "Interest Rate": [bank_interest],
                    "EMI/Month": [emi_amount_per_month],
                    "EMI Tenure": [emi_tenure]
                }
                df = pd.DataFrame(data)
                text4 = "-Invoice-"
                formatted_text = f"\033[1m{text4.center(80)}\033[0m"
                print(formatted_text)
                print(df)
            else:
                print("No matching Emi data found for the selected car.")
        else:
            print("No purchase history found with EMI payment method.")
        connection.close()
    except Exception as e:
        print(f'An error occurred: {str(e)}')

if __name__ == "__main__":
    main()

No purchase history found.
